# Word2Vec (Word Embedding)

Implement Word2Vec algorithm to compute vector representation of words, with Tensorflow 2.0. This example is using a small chunk of Wikipedia to train from.

More info: Mikolov, Tomas et al. "Efficient Estimation of Word Representations in Vector Space.", 2013

In [ ]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import random
import urllib
import zipfile
import collections

import numpy as np
import tensorflow as tf

In [ ]:
from typing import List

# Training Parameters
learning_rate: float = 0.1
batch_size: int = 128
num_steps: int = 3000000
display_step: int = 10000
eval_step: int = 200000

# Evaluate Parameters
eval_words: List[str] = ["five", "of", "going", "hardware", "american", "britain"]

# Word2Vec Parameters
embedding_size: int = 200 # Dimension of the embedding vector
max_vocabulary_size: int = 50000 # Total number of different words in the vocabulary
min_occurence: int = 10 # Remove all words that does not appear at least n times
skip_window: int = 3 # How many times to consider left and right
num_skips: int = 2 # How many times to reuse an input to generate a label
num_sampled: int = 64 # Number of negative example to sample

In [ ]:
# Download a small chunk of Wikipedia articles collection.
url: str = 'http://mattmahoney.net/dc/text8.zip'
data_path = "text8.zip"

test_words: str

if not os.path.exists(data_path):
  print("Downloading the dataset... (It may take some time)")
  filename, _ = urllib.request.urlretrieve(url, data_path)
  print("Done!")

# Unzip the dataset file. Text has already been processed
with zipfile.ZipFile(data_path) as f:
  text_words = f.read(f.namelist()[0]).lower().split()

Done!


In [ ]:
# Build the dictionary and replace rare words with UNK token
count = [("UNK", -1)]
# Retrieve the most common words
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
# Remove samples with less than 'min_occurence' occurences
for i in range(len(count) -1, -1, -1):
  if count[i][1] < min_occurence:
    count.pop(i)
  else:
    # Retrieve a word if, or assign it index 0 ("UNK") if not in dictionary
    break
# Compute the vocabulary size.
vocabulary_size = len(count)
# Assign an id to each word
word2id = dict()
for i, (word, _) in enumerate(count):
  try:
    if len(word.decode("utf8")) > 1:
      word2id[word.decode("utf8")] = i
  except Exception as e:
    print("Not a byte")
  else:
    word2id[word] = i

data = list()
unk_count = 0

for word in text_words:
  # Retrieve a word id or assign it index 0 ('UNK') if not in dictionary
  index = word2id.get(word, 0)
  if index == 0:
    unk_count = unk_count + 1
  data.append(index)

count[0] = ("UNK", unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Not a byte
Words count: 17005207
Unique words: 253854
Vocabulary size: 47135
Most common words: [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [ ]:
data_index: int = 0
# Generate training batch for the skip-gram model
def next_batch(batch_size: int, num_skips: int, skip_window: int):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window

  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  # get window size (words left and right and current one)
  span = 2 * skip_window + 1
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index = data_index + span

  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]

      if data_index == len(data):
        buffer.extend(data[0:span])
        data_index = span
      else:
        buffer.append(data[data_index])
        data_index = data_index + 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [ ]:
# Ensure the following ops & var are assigned on GPU
# (some ops are nmot compatible on GPU)

with tf.device("/cpu:0"):
  # Create the embedding varaible (each row represent a word embedding vector)
  embedding = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
  # Construct the varibales for the NCE loss
  nce_weights = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
  nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

def get_embedding(x):
  with tf.device("/cpu:0"):
    # Lookup the corresponding embedding vectors for each sample in X
    x_embed = tf.nn.embedding_lookup(embedding, x)
    return x_embed

def nce_loss(x_embed, y):
  with tf.device("/cpu:0"):
    y = tf.cast(y, tf.int64)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=y,
                       inputs=x_embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size)
    )

    return loss

# Evaluation
def evaluate(x_embed):
  with tf.device("/cpu:0"):
    # Compute the cosine similarity between input data embedding and very embedding vectors
    x_embed = tf.cast(x_embed, tf.float32)
    x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
    embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims = True))
    cosine_sim_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)

    return cosine_sim_op
  
# Define the optimizer
optimizers = tf.optimizers.SGD(learning_rate)

In [ ]:
# Optimization process
def run_optimization(x, y):
  with tf.device("/cpu:0"):
    # Wrap computation inside a GradientTape for automatic differentiation
    with tf.GradientTape() as g:
      emb = get_embedding(x)
      loss = nce_loss(emb, y)
    
    # Compute gradients
    gradients = g.gradient(loss, [embedding, nce_weights, nce_biases])

    # Update W and b following gradients
    optimizers.apply_gradients(zip(gradients, [embedding, nce_weights, nce_biases]))

In [ ]:
# Words for testing
x_test = np.array([word2id[w] for w in eval_words])
# Run training for the given number of steps
for step in range(1, num_steps+1):
  batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
  run_optimization(batch_x, batch_y)

  if step % display_step == 0 or step == 1:
    loss = nce_loss(get_embedding(batch_x), batch_y)
    print(f"steps: {step}, loss: {loss}")

    # Evaluation
  if step % eval_step == 0 or step == 1:
    print("Evaluation...")
    sim = evaluate(get_embedding(x_test)).numpy()
    for i in range(len(eval_words)):
      top_k = 8  # number of nearest neighbors.
      nearest = (-sim[i, :]).argsort()[1:top_k + 1]
      log_str = f"{eval_words[i]} nearest neighbors"
      for k in range(top_k):
        try:
          tmp = id2word[nearest[k]].decode()
          log_str = '{0} {1},'.format(log_str, tmp)
        except Exception as e:
          # print(e.__repr__())
          pass
        else:
          pass
      print(log_str)
    


steps: 1, loss: 537.4056396484375
Evaluation...
five nearest neighbors practiced, distress, concerning, montpellier, chs, werke, shipboard, bun,
of nearest neighbors cathedrals, technicality, incarnation, personhood, flutter, ceramic, planet, gygax,
going nearest neighbors alhazred, refering, messaging, arranges, pizarro, shatila, overrun, deirdre,
hardware nearest neighbors crowley, scantily, swearing, nanjing, nongovernmental, intruders, esso, cultivar,
american nearest neighbors slashdot, presque, bowing, antlers, louise, depicts, horses, monopoly,
britain nearest neighbors peasant, xxvii, symmetry, barton, bah, rainwater, espouses, modest,
steps: 10000, loss: 146.65516662597656
steps: 20000, loss: 40.09314727783203
steps: 30000, loss: 76.89292907714844
steps: 40000, loss: 40.19158935546875
steps: 50000, loss: 49.35409164428711
steps: 60000, loss: 20.357053756713867
steps: 70000, loss: 28.436458587646484
steps: 80000, loss: 33.27979278564453
steps: 90000, loss: 35.885292053222656
st